In [1]:
pip install chart_studio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 64 kB 2.0 MB/s 
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11447 sha256=0ca0056387e61ab323c64df1992a668bb8305e2fe126b20e8e79e258c959ea44
  Stored in directory: /root/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying


In [2]:
import chart_studio

In [3]:
import seaborn as sns
import pandas as pd
import plotly.graph_objects as go
import chart_studio.plotly as py
import plotly
import json, urllib
import numpy as np

In [18]:
# raw data: https://gist.github.com/nicolasesnis/eb3b35545e97926ab53e0617c5e4b639

data = pd.read_csv('raw.csv')[['user_id', 'time_install', 'event_name', 'time_event']]

# there are 4 columns in the original data frame:
# user_id (string): unique identifier of a user who installed the Mobile Application
# time_install (datetime): the time when the user installed the Mobile Application (1 time_install per user_id)
# event_name (string): the name of a specific in-app event (more event_name per user_id) 
# 3 unique values available: purchase, signup, reopen
# time_event (datetime): the time of each event (1 time_event per user_id)

data.head(10)

,user_id,time_install,event_name,time_event
0,4c6065c9466bc68e324e316edfb0227ff7cccc6c,43892.978137,purchase,43892.983808
1,f3049eac4788ffd4482390f8333d7e1adbf4c5a1,43896.220370,signup,43896.220799
2,f3049eac4788ffd4482390f8333d7e1adbf4c5a1,43896.220370,purchase,43896.221887
3,9c1e35e89a374207409ad05da6d69d43f427c5f2,43896.788056,reopen,43896.107708
4,723fba1295b9a7c8321bbc433f87629b90660582,43892.000046,reopen,43893.577488
5,723fba1295b9a7c8321bbc433f87629b90660582,43892.000046,signup,43892.002523
6,97963ab19ae86316d639e3a181a156c6c9dd5be0,43892.910023,purchase,43893.002616
7,97963ab19ae86316d639e3a181a156c6c9dd5be0,43892.910023,signup,43892.913715
8,97963ab19ae86316d639e3a181a156c6c9dd5be0,43892.910023,purchase,43892.971169
9,3f299029526b58c482521f7ad6fa907eca31063b,43894.194329,purchase,43894.198067


In [19]:
# unique user ids in original provided file is 25 803
data['user_id'].nunique() 

25803

In [21]:
# number of rows in original provided file is 50 356
len(data)

50356

In [22]:
# convert to datetime types:
data['time_event'] = pd.to_datetime(data['time_event'])
data['time_install'] = pd.to_datetime(data['time_install'])

# limit event_time by time_install, some rows and user IDs were removed
data = data[data.time_event >= data.time_install]

In [24]:
# unique user ids in frame is 25 791
data['user_id'].nunique() 

25791

In [25]:
# number of rows in frame is 50 285
len(data)

50285

In [26]:
# Edit this dataframe so that installs are passed as events

# Create a new DF from the data DF containing only install data
installs = data[['user_id', 'time_install']].sort_values(
    'time_install').drop_duplicates('user_id')

# Create an install column containing dummy "install" events
installs['event_name'] = 'install'

# Create an event_type column to keep the information of install vs other events
installs['event_type'] = 'install'

# Rename time_install to time_event
installs.rename(columns={'time_install': 'time_event'}, inplace=True)

# In the data DF, keep only events data and create the event_type column

data = data[['user_id', 'event_name',
             'time_event']].drop_duplicates()
data['event_type'] = 'in_app_action'

# Concatenate the two DataFrames
data = pd.concat([data, installs[data.columns]])

In [27]:
# unique user ids in frame is 25 791
data['user_id'].nunique() 

25791

In [28]:
# number of rows in frame increased to 68 394, users with install event type were added from columns to rows
len(data)

68394

In [29]:
data.head()

,user_id,event_name,time_event,event_type
0,4c6065c9466bc68e324e316edfb0227ff7cccc6c,purchase,1970-01-01 00:00:00.000043892,in_app_action
1,f3049eac4788ffd4482390f8333d7e1adbf4c5a1,signup,1970-01-01 00:00:00.000043896,in_app_action
2,f3049eac4788ffd4482390f8333d7e1adbf4c5a1,purchase,1970-01-01 00:00:00.000043896,in_app_action
3,9c1e35e89a374207409ad05da6d69d43f427c5f2,reopen,1970-01-01 00:00:00.000043896,in_app_action
4,723fba1295b9a7c8321bbc433f87629b90660582,reopen,1970-01-01 00:00:00.000043893,in_app_action


In [30]:
# Based on the time of events, we can compute the rank of each action at the user_id level:

# a) Sort ascendingly per user_id and time_event
# sort by event_type to make sure installs come first

data.sort_values(['user_id', 'event_type', 'time_event'],
                 ascending=[True, False, True], inplace=True)

# b) Group by user_id
grouped = data.groupby('user_id')

# c) Define a ranking function based on time_event, using the method = 'first' param to ensure no events have the same rank

def rank(x): return x['time_event'].rank(method='first').astype(int)


# d) Apply the ranking function to the data DF into a new "rank_event" column

data["rank_event"] = grouped.apply(rank).reset_index(0, drop=True)

# Add, each row, the information about the next_event

# a) Regroup by user_id

grouped = data.groupby('user_id')

# b) The shift function allows to access the next row's data. Here, we'll want the event name


def get_next_event(x): return x['event_name'].shift(-1)


# c) Apply the function into a new "next_event" column

data["next_event"] = grouped.apply(
    lambda x: get_next_event(x)).reset_index(0, drop=True)

In [31]:
# number of rows in frame is 68 394
len(data)

68394

In [32]:
# unique user ids in frame is 25 791
data['user_id'].nunique() 

25791

In [33]:
# display the orignal column (event_type) and the new column (next_event) 
# new event_name install has been created from time_install timestamp, instead of 3 event names we have 4 
data.head()

,user_id,event_name,time_event,event_type,rank_event,next_event
36344,0001e4ca097cbf23febc8fa9f0836e7d92fadb1a,install,1970-01-01 00:00:00.000043895,install,1,signup
36344,0001e4ca097cbf23febc8fa9f0836e7d92fadb1a,signup,1970-01-01 00:00:00.000043895,in_app_action,2,NaN
37585,000332c9fcf184c67347e57f19789f30cf3784d6,install,1970-01-01 00:00:00.000043892,install,1,signup
37585,000332c9fcf184c67347e57f19789f30cf3784d6,signup,1970-01-01 00:00:00.000043892,in_app_action,2,reopen
37586,000332c9fcf184c67347e57f19789f30cf3784d6,reopen,1970-01-01 00:00:00.000043894,in_app_action,3,NaN


In [34]:
# compute the time difference - time from each event to its next event

# a) regroup by user_id

grouped = data.groupby('user_id')

# b) use the shift function:

def get_time_diff(
    x): return x['time_event'].shift(-1) - x['time_event']

# c) crete new column  "time_to_next" 

data["time_to_next"] = grouped.apply(
    lambda x: get_time_diff(x)).reset_index(0, drop=True)

In [35]:
# unique user ids in frame is 25 791
data['user_id'].nunique() 

25791

In [36]:
# number of rows in frame is 68 394
len(data)

68394

In [37]:
# display the new column with time difference (time_to_next)
data.head()

,user_id,event_name,time_event,event_type,rank_event,next_event,time_to_next
36344,0001e4ca097cbf23febc8fa9f0836e7d92fadb1a,install,1970-01-01 00:00:00.000043895,install,1,signup,0 days 00:00:00
36344,0001e4ca097cbf23febc8fa9f0836e7d92fadb1a,signup,1970-01-01 00:00:00.000043895,in_app_action,2,NaN,NaT
37585,000332c9fcf184c67347e57f19789f30cf3784d6,install,1970-01-01 00:00:00.000043892,install,1,signup,0 days 00:00:00
37585,000332c9fcf184c67347e57f19789f30cf3784d6,signup,1970-01-01 00:00:00.000043892,in_app_action,2,reopen,0 days 00:00:00.000000002
37586,000332c9fcf184c67347e57f19789f30cf3784d6,reopen,1970-01-01 00:00:00.000043894,in_app_action,3,NaN,NaT


In [38]:
# prepare data frame to plot the journey up to the 20th action. This can be achieved by filtering the dataframe based on the rank_event column that we computed:
data = data[data.rank_event < 20]

# Check that you have only installs at rank 1:
data[data['rank_event'] == 1].event_name.unique()

# Working on the nodes_dict

all_events = list(data.event_name.unique())

# create color hex palette for plot
palette = ['808A87', '1E1E1E', 'EBEBEB','CDAF95', '000000', 'EEEEE0', 'FAFAFA']

# convert palette from hex to rgb 
for i, col in enumerate(palette):
    palette[i] = tuple(int(col[i:i+2], 16) for i in (0, 2, 4))

# append a seaborn complementary palette 
complementary_palette = sns.color_palette(
    "deep", len(all_events) - len(palette))
if len(complementary_palette) > 0:
    palette.extend(complementary_palette)

output = dict()
output.update({'nodes_dict': dict()})

i = 0
for rank_event in data.rank_event.unique():  # For each rank of event...
    # Create a new key equal to the rank...
    output['nodes_dict'].update(
        {rank_event: dict()}
    )

    # Look at all the events that were done at this step of the funnel...
    all_events_at_this_rank = data[data.rank_event ==
                                   rank_event].event_name.unique()

    # Read the colors for these events and store them in a list...
    rank_palette = []
    for event in all_events_at_this_rank:
        rank_palette.append(palette[list(all_events).index(event)])

    # Keep trace of the events names, colors and indices.
    output['nodes_dict'][rank_event].update(
        {
            'sources': list(all_events_at_this_rank),
            'color': rank_palette,
            'sources_index': list(range(i, i+len(all_events_at_this_rank)))
        }
    )
    # Finally, increment by the length of this ranks available events to make sure next indices will not be chosen from existing ones
    i += len(output['nodes_dict'][rank_event]['sources_index'])

# Working on the links_dict

output.update({'links_dict': dict()})

# group the DataFrame by user_id and rank_event

grouped = data.groupby(['user_id', 'rank_event'])

# define a function to read the sources, targets, values and time from event to next_event:

def update_source_target(user):
    try:
        source_index = output['nodes_dict'][user.name[1]]['sources_index'][output['nodes_dict']
                                                                           [user.name[1]]['sources'].index(user['event_name'].values[0])]

        target_index = output['nodes_dict'][user.name[1] + 1]['sources_index'][output['nodes_dict']
                                                                               [user.name[1] + 1]['sources'].index(user['next_event'].values[0])]

        if source_index in output['links_dict']:
            if target_index in output['links_dict'][source_index]:

                output['links_dict'][source_index][target_index]['unique_users'] += 1
                output['links_dict'][source_index][target_index]['avg_time_to_next'] += user['time_to_next'].values[0]
            else:

                output['links_dict'][source_index].update({target_index:
                                                           dict(
                                                               {'unique_users': 1,
                                                                'avg_time_to_next': user['time_to_next'].values[0]}
                                                           )
                                                           })
        else:

            output['links_dict'].update({source_index: dict({target_index: dict(
                {'unique_users': 1, 'avg_time_to_next': user['time_to_next'].values[0]})})})
    except Exception as e:
        pass

In [39]:
# number of rows in frame is 68 394
len(data)

68394

In [40]:
# unique user ids in frame is 25 791
data['user_id'].nunique() 

25791

In [41]:
# see the rgb palette instead of hexadecimal
palette

[(128, 138, 135),
 (30, 30, 30),
 (235, 235, 235),
 (205, 175, 149),
 (0, 0, 0),
 (238, 238, 224),
 (250, 250, 250)]

In [42]:
# print the data frame until the 20th event step
data.head()

,user_id,event_name,time_event,event_type,rank_event,next_event,time_to_next
36344,0001e4ca097cbf23febc8fa9f0836e7d92fadb1a,install,1970-01-01 00:00:00.000043895,install,1,signup,0 days 00:00:00
36344,0001e4ca097cbf23febc8fa9f0836e7d92fadb1a,signup,1970-01-01 00:00:00.000043895,in_app_action,2,NaN,NaT
37585,000332c9fcf184c67347e57f19789f30cf3784d6,install,1970-01-01 00:00:00.000043892,install,1,signup,0 days 00:00:00
37585,000332c9fcf184c67347e57f19789f30cf3784d6,signup,1970-01-01 00:00:00.000043892,in_app_action,2,reopen,0 days 00:00:00.000000002
37586,000332c9fcf184c67347e57f19789f30cf3784d6,reopen,1970-01-01 00:00:00.000043894,in_app_action,3,NaN,NaT


In [43]:
# apply the function to your grouped object
grouped.apply(lambda user: update_source_target(user))


targets = []
sources = []
values = []
time_to_next = []

for source_key, source_value in output['links_dict'].items():
    for target_key, target_value in output['links_dict'][source_key].items():
        sources.append(source_key)
        targets.append(target_key)
        values.append(target_value['unique_users'])
        time_to_next.append(str(pd.to_timedelta(
            target_value['avg_time_to_next'] / target_value['unique_users'])).split('.')[0])  # Split to remove the milliseconds information

labels = []
colors = []
for key, value in output['nodes_dict'].items():
    labels = labels + list(output['nodes_dict'][key]['sources'])
    colors = colors + list(output['nodes_dict'][key]['color'])

for idx, color in enumerate(colors):
    colors[idx] = "rgb" + str(color) + ""

# plot the data until the 20th event in sankey diagram decision tree

fig = go.Figure(data=[go.Sankey(
    node=dict(
        thickness=10, 
        line=dict(color="black", width=0.5),
        label=labels,
        color=colors
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,
        label=time_to_next,
        hovertemplate='%{value} unique users went from %{source.label} to %{target.label}.<br />' +
        '<br />It took them %{label} in average.<extra></extra>',
    ))])

fig.update_layout(autosize=True, title_text="Application Events in Funnel Stream", font=dict(size=10), plot_bgcolor='white')

In [44]:
pip install kaleido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79.9 MB 151 kB/s 


In [45]:
import kaleido

In [46]:
# save the tree node diagram to jpg
from kaleido.scopes.plotly import PlotlyScope
import plotly.graph_objects as go
scope = PlotlyScope(
    plotlyjs="https://cdn.plot.ly/plotly-latest.min.js")


with open("journey.jpg", "wb") as f:
    f.write(scope.transform(fig, format="jpg"))

In [47]:
# create sankey in matplotlib

In [48]:
pip install sankey

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sankey: filename=sankey-0.0.2-py3-none-any.whl size=10702 sha256=e14331cab21c48c2fd86149b1dcb19ac104bd8e40e281f792158b72e58a97401
  Stored in directory: /root/.cache/pip/wheels/7d/12/43/ba429e9b91188667b08f79274f4181ed823c91dfffe766c676
Successfully built sankey


In [49]:
pip install dash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.8 MB 5.3 MB/s 
     |████████████████████████████████| 357 kB 65.6 MB/s 


In [50]:
import dash

In [51]:
import sankey
from sankey import *
# Import the sankey function from the sankey module within pySankey
from matplotlib.sankey import Sankey
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
pd.options.display.max_rows=8

In [52]:
df = data[['event_name', 'next_event', 'user_id']].copy()

In [53]:
df['distinct_count'] = df.groupby(['event_name'])['user_id'].transform('nunique')

In [54]:
df1 = df.groupby('event_name')
df2 = df1['next_event'].value_counts()

In [55]:
pd.set_option('display.max_rows', None)

In [56]:
df2

event_name  next_event
install     signup        12342
            reopen         6746
            purchase       6703
purchase    reopen         2972
            signup         2701
            purchase        594
reopen      reopen         2802
            purchase       1082
            signup          169
signup      reopen         3880
            purchase       2596
            signup           16
Name: next_event, dtype: int64

In [58]:
# number of rows in df frame is 68 394
len(df)

68394

In [57]:
# unique user ids in df frame is 25 791
df['user_id'].nunique() 

25791

In [59]:
# data

label = ["install", "signup", "reopen", "purchase"]
source = [0, 0, 0,        # install
          1, 1, 1,        # signup
          2, 2, 2,        # reopen
          3, 3, 3 ]       # purchase

target = [3, 2, 1, 
          3, 2, 1,
          3, 2, 1, 
          3, 2, 1 ]

value = [6703, 6746, 12342,  
         593, 2969, 2701,
         1080, 2801,169, 
         2596, 3880, 16]

color_node = [
'#808A87', '#808A87', '#808A87', 
'#1E1E1E', '#1E1E1E', '#1E1E1E',
'#EBEBEB', '#EBEBEB', '#EBEBEB', 
'#CDAF95', '#CDAF95', '#CDAF95']

color_link = [
'#808A87', '#808A87', '#808A87', 
'#1E1E1E', '#1E1E1E', '#1E1E1E',
'#EBEBEB', '#EBEBEB', '#EBEBEB', 
'#CDAF95', '#CDAF95', '#CDAF95']

# data to dict, dict to sankey
link = dict(source = source, target = target, value = value, color=color_link)
node = dict(label = label, pad=50, thickness=5, color=color_node)

data = go.Sankey(link = link, node=node)
# plot
fig = go.Figure(data)
fig.update_layout(
    hovermode = 'x',
    title="Application Basic Flow", font=dict(size=10), plot_bgcolor='white'
)
fig.show()


In [60]:
with open("flow.jpg", "wb") as f:
    f.write(scope.transform(fig, format="jpg"))